In [470]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import pickle
from sklearn import linear_model
import re
from sklearn.model_selection import train_test_split

In [471]:
mode_dict = {'brand': 'ASUS',
             'processor_brand': 'Intel',
             'processor_name': 'Core i5',
             'processor_gnrtn': '11th',
             'ram_gb': '8 GB',
             'ram_type': 'DDR4',
             'ssd': '512 GB',
             'hdd': '0 GB',
             'os': '64-bit Windows',
             'graphic_card_gb': '0 GB',
             'weight': 'Casual',
             'warranty': '1 year',
             'Touchscreen': 'No',
             'msoffice': 'No',
             'Price': 77031,
             'rating': 'Good Rating',
             'Number of Ratings': 0,
             'Number of Reviews': 0}


In [472]:
#========================================================== Helper Functions ========================================================================

# Load Model
def load(file_path):
    loaded_model = pickle.load(open(file_path, 'rb'))
    return loaded_model



# Extract number from string
def extract_number(number_string, mode_value=0):
    if isinstance(number_string, str):
        match = re.search(r'\d+', number_string)  # Find the first occurrence of one or more digits
        if match:
            return int(match.group())  # Convert the matched digits to an integer
        else:
            return mode_value  # If no digits found, return 0
    else:
        return number_string


# Fill nulls with mode value
def fill_nulls(data):
     for col, mode in mode_dict.items():
        data[col].fillna(mode, inplace=True)

# Handle Missing Values
def remove_outliers(data):

    lower_bound = {'brand': -5.0, 'processor_name': -2.5, 'processor_gnrtn': 8.5, 'ram_gb': -2.0, 'ram_type': 3.0, 'ssd': -128.0, 'hdd': -768.0, 'graphic_card_gb': -3.0, 'weight': -0.5, 'warranty': -1.5, 'Touchscreen': 0.0, 'msoffice': -1.5, 'Price': -19869.0, 'Number of Ratings': -231.0, 'Number of Reviews': -29.25, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': -1.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}
    upper_bound = {'brand': 11.0, 'processor_name': 9.5, 'processor_gnrtn': 12.5, 'ram_gb': 14.0, 'ram_type': 3.0, 'ssd': 896.0, 'hdd': 1280.0, 'graphic_card_gb': 5.0, 'weight': 3.5, 'warranty': 2.5, 'Touchscreen': 0.0, 'msoffice': 2.5, 'Price': 155675.0, 'Number of Ratings': 385.0, 'Number of Reviews': 48.75, 'os_bits': 64.0, 'AMD': 0.0, 'Intel': 2.5, 'M1': 0.0, 'DOS': 0.0, 'Mac': 0.0, 'Windows': 1.0}

    s = ['ram_type' ,'Touchscreen', 'os_bits' , 'AMD' , 'M1' , 'DOS' , 'Mac' , 'Windows' ]

    # Replace the outliers with lower_bound or upper_bound values
    for col in data.iloc[:,:-1]:
        if col not in s:
            data.loc[data[col] < lower_bound[col], col] = lower_bound[col]
            data.loc[data[col] > upper_bound[col], col] = upper_bound[col]

    return data


# Feature selection using chi-squared
def feature_selection(data):
    selected_indices = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
    data = data.iloc[: , selected_indices]
    return data

# Execute Model
def execute_model(model, X_test, Y_test, model_name):
    Y_test_pred = model.predict(X_test)
    test_mse = mean_squared_error(Y_test, Y_test_pred)
    print(f" Testing MSE: {test_mse}")
    test_accuracy = accuracy_score(Y_test, Y_test_pred)
    print(f" Testing Accuracy: {test_accuracy}")


In [473]:
# Preprocess Data
def preprocess_data(data):
    # filling nulls
    fill_nulls(data)


    # processor_gnrtn
    # replace not available with mode
    data['processor_gnrtn'].replace('Not Available', '11th', inplace = True)

    # extract number from string else replace sith mode
    data['processor_gnrtn'] = data['processor_gnrtn'].apply(lambda x: extract_number(x, 11))

    # ram_gb
    # extract number from string else replace sith mode
    data['ram_gb'] = data['ram_gb'].apply(lambda x: extract_number(x, 8))

    # ssd
    # extract number from string else replace sith mode
    data['ssd'] = data['ssd'].apply(lambda x: extract_number(x, 512))

    # hdd
    # extract number from string else replace sith mode
    data['hdd'] = data['hdd'].apply(lambda x: extract_number(x, 0))


    # os
    # split os column to two columns
    data[['os_bits', 'os_type']] = data['os'].str.split(' ', expand=True)
    data = data.drop('os', axis = 1)

    # extract number from string else replace sith mode
    data['os_bits'] = data['os_bits'].apply(lambda x: extract_number(x, 64))

    # graphic_card_gb
    # extract number from string else replace sith mode
    data['graphic_card_gb'] = data['graphic_card_gb'].apply(lambda x: extract_number(x, 0))


    # warranty
    # extract number from string else replace sith mode
    data['warranty'] = data['warranty'].apply(lambda x: extract_number(x, 1))


    # Touchscreen
    touch_screen_mapping = {
        'No': 0,
        'Yes': 1
    }

    # map each value else replace with mode
    data['Touchscreen'] = data['Touchscreen'].map(touch_screen_mapping).fillna(0)




    # msoffice
    msoffice_screen_mapping = {
        'No': 0,
        'Yes': 1
    }

    # map each value else replace with mode
    data['msoffice'] = data['msoffice'].map(msoffice_screen_mapping).fillna(0)


    # rating
    rating_performance_mapping = {
        'Bad Rating': 0,
        'Good Rating': 1
    }

    # map each value else replace with mode
    data['rating'] = data['rating'].map(rating_performance_mapping)

    # ram_type
    ram_performance_mapping = {
        'LPDDR4X': 5,
        'LPDDR4': 4,
        'DDR4': 3,
        'DDR3': 2,
        'DDR5': 6,
        'LPDDR3': 1
    }

    data['ram_type'] = data['ram_type'].map(ram_performance_mapping).fillna(3)
    # data['ram_type'].value_counts()

    # weight
    weight = {
        'Casual':2,
        'ThinNlight':1,
        'Gaming':3,
    }

    # map each value else replace with mode
    data['weight'] = data['weight'].map(weight).fillna(2)


    brand_mapping = {
    'ASUS': 1,
    'Lenovo': 5,
    'HP': 4,
    'APPLE': 0,
    'DELL': 3,
    'acer': 7,
    'MSI': 6,
    'Avita': 2
    }

    # map each value else replace with mode
    data['brand'] = data['brand'].map(brand_mapping).fillna(1)


    processor_name_mapping = {
    'Pentium Quad': 6,
    'Core i5': 2,
    'Celeron Dual': 0,
    'Core i3': 1,
    'Ryzen 7': 9,
    'M1': 5,
    'Ryzen 5': 8,
    'Core i7': 3,
    'Ryzen 3': 7,
    'Ryzen 9': 10,
    'Core i9': 4
    }

    # map each value else replace with mode
    data['processor_name'] = data['processor_name'].map(processor_name_mapping).fillna(2)


    # Apply One-Hot Encoders to data
    # processor brand
    data['processor_brand'] = data['processor_brand'].apply(lambda x: x if x in ['AMD', 'Intel', 'M1'] else 'Intel')

    one_hot_encoded = pd.get_dummies(data['processor_brand'])
    one_hot_encoded = one_hot_encoded.astype(int)


    data = data.drop('processor_brand', axis = 1)
    data = pd.concat([data, one_hot_encoded], axis=1)




    missing_column = 'AMD'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'Intel'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'M1'
    if missing_column not in data.columns:
        data[missing_column] = 0


    # os_type
    data['os_type'] = data['os_type'].apply(lambda x: x if x in ['DOS', 'Mac', 'Windows'] else 'Windows')


    one_hot_encoded = pd.get_dummies(data['os_type'])
    one_hot_encoded = one_hot_encoded.astype(int)
    data = data.drop('os_type', axis = 1)
    data = pd.concat([data, one_hot_encoded], axis=1)


    missing_column = 'DOS'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'Mac'
    if missing_column not in data.columns:
        data[missing_column] = 0

    missing_column = 'Windows'
    if missing_column not in data.columns:
        data[missing_column] = 0


    # Replace target column
    rating_col = data['rating']
    data = data.drop('rating', axis = 1)
    data = pd.concat([data, rating_col], axis=1)

    data = remove_outliers(data)


    return data

In [474]:
    # Load Model
    decision_tree_model = load('ms2 model classification.sav')


    # Load Test Data
    test_data = pd.read_csv('/content/ElecDeviceRatingPrediction_Milestone2.csv')


    # Preprocess Test Data
    test_data = preprocess_data(test_data)

    # Ensure test data has the same features as training data
    X_test = test_data[['brand', 'processor_name', 'processor_gnrtn', 'ram_gb', 'ram_type',
       'ssd', 'hdd', 'graphic_card_gb', 'weight', 'warranty', 'Touchscreen',
       'msoffice', 'Price', 'Number of Ratings', 'Number of Reviews',
       'os_bits', 'AMD', 'Intel', 'M1', 'DOS', 'Mac', 'Windows', 'rating']]


    X_test = X_test.iloc[:, :-1]
    # Execute Model
    Y_test = test_data['rating']

    print(test_data)
    execute_model(decision_tree_model, X_test, Y_test, "Decision Tree")


     brand  processor_name  processor_gnrtn  ram_gb  ram_type  ssd  hdd  \
0        1             6.0             11.0       4         5    0  512   
1        1             2.0             10.0       8         4    0  512   
2        1             0.0             11.0       4         3    0  512   
3        5             1.0             11.0       4         3  256    0   
4        4             9.0             11.0      14         3  512    0   
..     ...             ...              ...     ...       ...  ...  ...   
718      5             1.0             11.0       4         3  256    0   
719      4             1.0             11.0       8         3  512    0   
720      5             2.0             11.0       8         3  512    0   
721      6             2.0             10.0       8         4    0  512   
722      1             1.0             11.0       8         3  512    0   

     graphic_card_gb  weight  warranty  ...  Number of Ratings  \
0                  0     2.0     